In [1]:
import selenium
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.keys import Keys

In [67]:
#크롤링 시작 함수
def starting(URL):
    driver = webdriver.Chrome(executable_path='chromedriver') #크롬 브라우저 사용
    driver.get(url=URL) #url 접속
    driver.implicitly_wait(time_to_wait=5) #페이지 로딩 기다리기
    #select = Select(driver.find_element_by_id('view-count1'))
    #select.select_by_value(value='15')
    return driver

#크롤링 끝내는 함수
def ending(dri):
    driver = dri
    driver.close()

#총 페이지 수 구하는 함수
def pageSetting(mypath):
    pagenv = driver.find_element_by_xpath(mypath) #해당 라벨 가져오기
    pagenum = pagenv.text
    pagenum = pagenum.split('/') #텍스트 분할
    return int(pagenum[1]) #정수로 변환해 반환

#(임시) 페이지 구하는 함수
def pageSetting2():
    pagenv = driver.find_elements_by_class_name('page_navi') #해당 라벨 가져오기
    pagenum = pagenv[0].text.split('\n')[2]
    pagenum = pagenum.split('/') #텍스트 분할
    return int(pagenum[1]) #정수로 변환해 반환

#페이지 넘기기
def pageOver(pagePath):
    posting = driver.find_element_by_xpath(pagePath)
    posting.click()

#페이지 처음으로
def pageBack(pagePath):
    posting = driver.find_element_by_xpath(pagePath)
    posting.click()
    
#페이지 크롤링하기
def pageCrowl():
    view = driver.find_elements_by_class_name('item_section') #원하는 라벨 가져오기
    arr = [] #텍스트 데이터를 저장할 빈 배열 생성
    for i in view: #텍스트 데이터 변환공식
        string = i.text
        arr.append(string.split('\n'))
    return arr #리스트를 반환

#별점 추출 함수
def star(arr):
    if '만점' in arr:
        return int(arr[-3:-2])
    else:
        return 0

# ,포함한 숫자 문자열 숫자 변환
def to_num(st):
    if ',' in st:
        return int(st.replace(',',''))
    else:
        return int(st)

#테그 배열 수 일치시키는 함수
def check_tag(arr):
    if len(arr) == 10:
        arr.insert(4,'')
        return arr
    else:
        return arr

#딕셔너리 형태로 반환하는 함수
def array_return(arr, n_data):
    dic = {}
    dic['장소'] = n_data
    if arr[0] == '':
        return dic
    else:
        for i in range(0, len(arr), 2):
            dic[arr[i]] = arr[i+1]
        return dic
    
#방문횟수 좋아요 등 부가 정보를 딕셔너리로 변환 하는 함수
def show_dic():
    m_data = driver.find_element_by_css_selector(
        '#content > div > div.cont_wrap.sub_visual > ul').text.split('\n') #조회관련
    dic = {}
    for i in range(0, len(m_data), 2):
        dic[m_data[i]] = to_num(m_data[i+1])
    return dic

#주변 데이터 사전타입 변환 함수
def linked_point_data(driver, name):
    driver.find_element_by_link_text('지도/주변관광지').send_keys(Keys.ENTER)
    time.sleep(0.5)
    fellow = driver.find_element_by_class_name('util_area')
    fellow_data = fellow.text.split('\n')
    dic = {}
    for i in fellow_data[1:]:
        driver.find_element_by_link_text(i).send_keys(Keys.ENTER)
        time.sleep(0.2)
        map_data = driver.find_elements_by_css_selector(
            '#tab2 > div.find_map_area > div > div.mapList')
        map_list = []
        map_list = map_data[0].text.split('\n')
        linked_map = []
        for j in range(0, len(map_list), 6):
            if map_list[j] == name:
                continue
            else:
                linked_map.append(map_list[j])
        dic[i.split('(')[0]] = linked_map
    return dic

In [68]:
#크롤링할 페이지
URL = 'https://www.visitjeju.net/kr/#'
driver = starting(URL) #드라이버 정보 저장하기
df = pd.DataFrame() #빈 데이터프레임 생성
count=0 #진행과정을 보기 위해서 만든 변수
for link_data in link_list:
    driver.get(link_data)
    driver.implicitly_wait(5)
    
    section = driver.find_element_by_class_name('p_depth').text.split('>')[1] #장소분류
    name = driver.find_element_by_class_name('sub_info_title').text.replace('\n','') #장소이름
    add = driver.find_element_by_class_name('info_sub_cont').text #장소주소
    hesh = driver.find_element_by_class_name('tag_area').text.split('\n') #장소 해시태그
    i_data = driver.find_elements_by_class_name('add2020_detail_side_info') #이용안내 데이터
    heshlist = [] #해시태그 저장할 배열
    for i in hesh: #해시태그 가공
        heshlist.extend(i.split('#')[1:])

    for i in i_data:
        i_arr = i.text.split('\n')

    m_data_dic = show_dic() #조회관련 사전타입 배열
    linked_point_dic = linked_point_data(driver, name) #주변포인트 반환타입:사전
    sub_dic = array_return(i_arr, name) #이용안내 데이터

    dit = {'타입':section,'장소':name,'위치':add,'해시태그':heshlist}
    dit.update(m_data_dic)
    dit.update(linked_point_dic)
    dit.update(sub_dic)

    df = df.append(dit, ignore_index=True)
    print(count)
    count+=1

ending(driver)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [7]:
URL = 'https://www.visitjeju.net/u/6Ko'

driver = starting(URL)

section = driver.find_element_by_class_name('p_depth').text.split('>')[1] #장소분류
print(section)
ending(driver)

In [11]:
#테마여행 링크 저장 코드
URL='https://www.visitjeju.net/kr/recommendTour?menuId=DOM_000002000000000211&region2cd=&contentscd=c6&listType=&tag=&andValue=&mjtorTag=&page=1'
driver = starting(URL) #드라이버 정보 저장하기
arr=[]
Select(driver.find_element_by_id('goto-size-1')).select_by_value(value='15')
pNum = pageSetting2()
for i in range(pNum):
    time.sleep(1)
    name_d = driver.find_elements_by_class_name('item_top')
    for k in name_d:
        li = k.find_element_by_tag_name('a').get_attribute('href')
        arr.append(li)
        print(arr[-1:])
    driver.find_element_by_link_text('다음 페이지').send_keys(Keys.ENTER)

ending(driver)

df = pd.DataFrame(arr)
df.to_csv('themelink.csv', encoding='CP949')

In [18]:
URL='https://www.visitjeju.net/kr/#'
driver = starting(URL)

df = pd.DataFrame(columns=['title','detail'])

for i in arr:
    time.sleep(1)
    driver.get(i)
    driver.implicitly_wait(5)
    title = driver.find_element_by_class_name('sub_info_title').text
    detail = driver.find_element_by_class_name('add2020_detail_box_in').text
    df = df.append({'title':title,'detail':detail}, ignore_index=True)

ending(driver)

df.to_csv('themeT.csv', encoding='UTF-8')

In [34]:
#기본정보 저장 코드(알파버전)
df = pd.DataFrame(columns=['category','name','location','tag','like','pick','review','rating'])

for j, i in enumerate(arr):
    i = check_tag(i)
    df = df.append({'category':i[0],
                   'name':i[2],
                   'location':i[3].split('> ')[1],
                   'tag':i[4].split('#')[1:],
                   'like':to_num(i[6]),
                   'pick':to_num(i[8]),
                   'review':to_num(i[10]),
                   'rating':star(i[1])}, ignore_index=True)
    print(j)
df.to_csv('숙박목록.csv', encoding='CP949')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [87]:
#세부정보 저장 코드(베타버전)
#크롤링할 페이지
URL = 'https://www.visitjeju.net/kr/#'
pagePath = '//*[@id="content"]/div/div[2]/div[5]/div[1]/div/div/div[4]/a[3]'
goFirstP = '//*[@id="content"]/div/div[2]/div[5]/div[1]/div/div/div[4]/a[1]'
mypath = '//*[@id="content"]/div/div[2]/div[5]/div[1]/div/div/div[4]/p'

driver = starting(URL) #드라이버 정보 저장하기
driver.implicitly_wait(10)
arr = [] #크롤링 데이터를 저장할 배열

#대분류 항목 구하기
c_name = driver.find_elements_by_class_name('list_gnb')
c_split_name = c_name[0].text.split('\n')

#중분류 항목 구하기
t_arr = driver.find_elements_by_class_name('inner_middle')
t_arr = t_arr[12].text
t_arr = t_arr.split('\n')
t_arr = t_arr[12:48]

#빈 데이터프레임 생성
df = pd.DataFrame(columns=['name','main_category','sub_category'])

#중분류 항목 정제
while '전체' in t_arr:
    t_arr.remove('전체') # '전체' 삭제
for i in c_split_name[:3]:
    t_arr.remove(i)

#대분류 순회하기
for i in t_arr:
    #driver.find_element_by_link_text(i).click() 가끔 click()함수가 안먹혀서 아래 코드 씀
    driver.find_element_by_link_text(i).send_keys(Keys.ENTER) #대분류 페이지 넘김
    driver.implicitly_wait(10) #혹시 모르는 페이지 로딩 대기
    s_arr = driver.find_elements_by_class_name('recommend_tour') #중분류 항목 구하기
    
    try: #중분류 항목이 없는 경우가 있어서 나눴음
        s_arr = s_arr[0].text.split('\n')
        if '전체' in s_arr:
            s_arr.remove('전체')
        if '그외' in s_arr:
            s_arr.remove('그외')
    except IndexError: #중분류 항목이 없는 경우 배열 삭제가 안되므로 오류로 빠짐
        pNum = pageSetting2() #페이지 html이 규칙적이지 못해 class명으로 검색하는 방법을 씀
        arr = [] #배열 초기화를 해야지 앞의 항목이 갱신 안 됨
        for n in range(pNum): #페이지 수만큼 반복
            time.sleep(1)
            n_arr = pageCrowl()
            arr.extend(n_arr)
            driver.find_element_by_link_text('다음 페이지').send_keys(Keys.ENTER)
        for m in arr: #만들어진 arr 배열 순회하면서 데이터프레임으로 저장
            df = df.append({'name':m[2],
                    'main_category':i,
                    'sub_category':''}, ignore_index=True)
            print(df.tail(1))
        driver.find_element_by_link_text('첫 페이지').send_keys(Keys.ENTER)
        continue
        
    for j in s_arr:
        #driver.find_element_by_link_text(j).click()
        driver.find_element_by_link_text(j).send_keys(Keys.ENTER)
        pNum = pageSetting(mypath) #페이지 수
        arr = []
        for k in range(pNum):
            time.sleep(1)
            n_arr = pageCrowl()
            arr.extend(n_arr)
            pageOver(pagePath)
        for l in arr:
            df = df.append({'name':l[2],
                    'main_category':i,
                    'sub_category':j}, ignore_index=True)
            print(df.tail(1))
        pageBack(goFirstP)
        
        time.sleep(1)

ending(driver)

df.to_csv('category_data.csv', encoding='UTF-8')

   name main_category sub_category
0  봄날카페            카페             
        name main_category sub_category
1  몽상드애월(몽상)            카페             
             name main_category sub_category
2  서연의집(영화 건축학개론)            카페             
   name main_category sub_category
3  리치망고            카페             
    name main_category sub_category
4  카페공작소            카페             
             name main_category sub_category
5  하하호호(HAHAHOHO)            카페             
   name main_category sub_category
6  우유부단            카페             
  name main_category sub_category
7  델문도            카페             
   name main_category sub_category
8  테라로사            카페             
    name main_category sub_category
9  마노르블랑            카페             
    name main_category sub_category
10  카페콜라            카페             
    name main_category sub_category
11  키친오즈            카페             
    name main_category sub_category
12  구좌상회            카페             
     name main_category sub_cate

170  커피가게쉬고가게            카페             
          name main_category sub_category
171  커피디셈버 함덕점            카페             
     name main_category sub_category
172  키아스마            카페             
     name main_category sub_category
173  팜알로하            카페             
     name main_category sub_category
174  픽스커피            카페             
      name main_category sub_category
175  하와유제주            카페             
          name main_category sub_category
176  한라산빵 1950            카페             
     name main_category sub_category
177  해적카페            카페             
      name main_category sub_category
178  헬로우우도            카페             
         name main_category sub_category
179  And유Café            카페             
             name main_category sub_category
180  Jeju in aA카페            카페             
       name main_category sub_category
181  가비오타카페            카페             
    name main_category sub_category
182  가온해            카페             
      name main_categ

336  이익새양과점            카페             
     name main_category sub_category
337  잔디가미            카페             
          name main_category sub_category
338  재게재게옵서 카페            카페             
       name main_category sub_category
339  제니의 정원            카페             
     name main_category sub_category
340  제리스펍            카페             
       name main_category sub_category
341  제주그림카페            카페             
      name main_category sub_category
342  제주돌창고            카페             
         name main_category sub_category
343  제주본초협동조합            카페             
            name main_category sub_category
344  제주샘주 영농협동조합            카페             
       name main_category sub_category
345  제주와이너리            카페             
     name main_category sub_category
346  제주의봄            카페             
          name main_category sub_category
347  제주한치빵 우도점            카페             
    name main_category sub_category
348  조아찌            카페             
      name main_cat

       name main_category sub_category
453  신라호텔제주          관광호텔             
        name main_category sub_category
454  롯데호텔 제주          관광호텔             
               name main_category sub_category
455  히든클리프 호텔 앤 네이쳐          관광호텔             
         name main_category sub_category
456  서귀포KAL호텔          관광호텔             
          name main_category sub_category
457  씨에스호텔앤리조트          관광호텔             
         name main_category sub_category
458  신라스테이 제주          관광호텔             
           name main_category sub_category
459  라마다프라자제주호텔          관광호텔             
         name main_category sub_category
460  롯데시티호텔제주          관광호텔             
         name main_category sub_category
461  메종글래드 제주          관광호텔             
           name main_category sub_category
462  제주부영호텔엔리조트          관광호텔             
         name main_category sub_category
463  루스톤빌라앤호텔          관광호텔             
        name main_category sub_category
464  제주KAL호텔          관광호텔             
  

613  호텔더원          관광호텔             
      name main_category sub_category
614  호텔빠레브          관광호텔             
      name main_category sub_category
615  호텔아로하          관광호텔             
        name main_category sub_category
616  호텔컬리넌제주          관광호텔             
       name main_category sub_category
617  홍익제주호텔          관광호텔             
     name main_category sub_category
618  화인호텔          관광호텔             
       name main_category sub_category
619  휴마루리조트          관광호텔             
          name main_category sub_category
620  씨에스호텔앤리조트       전통/가족호텔             
        name main_category sub_category
621  마레보 리조트       전통/가족호텔             
        name main_category sub_category
622  아름다운리조트       전통/가족호텔             
             name main_category sub_category
623  더포그레이스호텔앤리조트       전통/가족호텔             
           name main_category sub_category
624  베스트웨스턴제주호텔       전통/가족호텔             
      name main_category sub_category
625  일레인호텔       전통/가족호텔             
     

...

1167  블루마린         농어촌민박             
      name main_category sub_category
1168  블루오션         농어촌민박             
       name main_category sub_category
1169  비아 제주         농어촌민박             
       name main_category sub_category
1170  비오하우스         농어촌민박             
       name main_category sub_category
1171  비자낭달집         농어촌민박             
      name main_category sub_category
1172  비치조아         농어촌민박             
      name main_category sub_category
1173  비파민박         농어촌민박             
       name main_category sub_category
1174  빌라배알로         농어촌민박             
          name main_category sub_category
1175  뿌리게스트하우스         농어촌민박             
          name main_category sub_category
1176  쁘띠제주빅터펜션         농어촌민박             
        name main_category sub_category
1177  사계여행민박         농어촌민박             
       name main_category sub_category
1178  사계절민박         농어촌민박             
          name main_category sub_category
1179  사랑이꽃피는민박         농어촌민박             
      name m

1327  제주파크랜드         농어촌민박             
        name main_category sub_category
1328  제주팡숑예래         농어촌민박             
         name main_category sub_category
1329  제주펜션향림원         농어촌민박             
         name main_category sub_category
1330  제주프렌즈펜션         농어촌민박             
        name main_category sub_category
1331  제주필하우스         농어촌민박             
         name main_category sub_category
1332  제주해조대펜션         농어촌민박             
      name main_category sub_category
1333  조은민박         농어촌민박             
     name main_category sub_category
1334  조천댁         농어촌민박             
       name main_category sub_category
1335  조천스테이         농어촌민박             
       name main_category sub_category
1336  종달스토리         농어촌민박             
        name main_category sub_category
1337  중문햇빛펜션         농어촌민박             
       name main_category sub_category
1338  쥬빌리펜션         농어촌민박             
       name main_category sub_category
1339  지삿개풍경         농어촌민박             
      name m

     name main_category sub_category
1433  돌코롱            쇼핑          특산품
      name main_category sub_category
1434  베리제주            쇼핑          특산품
       name main_category sub_category
1435  뷰티풀제주            쇼핑          특산품
      name main_category sub_category
1436  삼다닷컴            쇼핑          특산품
       name main_category sub_category
1437  자연이야기            쇼핑          특산품
        name main_category sub_category
1438  제주민속식품            쇼핑          특산품
           name main_category sub_category
1439  (주)메이드인제주            쇼핑          특산품
     name main_category sub_category
1440  갈중이            쇼핑          특산품
                   name main_category sub_category
1441  농업회사법인 나무와열매 주식회사            쇼핑          특산품
               name main_category sub_category
1442  다나한한복 앤 러블리베베            쇼핑          특산품
      name main_category sub_category
1443  마켓오름            쇼핑          특산품
      name main_category sub_category
1444  명품수산            쇼핑          특산품
       name main_category sub_

1551  팰롱팰롱 빛나는            쇼핑        상점·상가
       name main_category sub_category
1552  펠롱잡화점            쇼핑        상점·상가
      name main_category sub_category
1553  풍년상회            쇼핑        상점·상가
        name main_category sub_category
1554  프레쉬모먼트            쇼핑        상점·상가
        name main_category sub_category
1555  피크닉메이트            쇼핑        상점·상가
      name main_category sub_category
1556  한라꽃방            쇼핑        상점·상가
     name main_category sub_category
1557  한복담            쇼핑        상점·상가


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/div/div[2]/div[5]/div[1]/div/div/div[4]/p"}
  (Session info: chrome=92.0.4515.107)


In [62]:
#스팟 세부정보 링크 저장 코드
#크롤링할 페이지
URL = 'https://www.visitjeju.net/kr/#'

driver = starting(URL) #드라이버 정보 저장하기
arr = [] #크롤링 데이터를 저장할 배열

c_name = driver.find_elements_by_class_name('list_gnb')
c_name = c_name[0].text.split('\n')[:4]

for i in c_name:
    driver.find_element_by_link_text(i).send_keys(Keys.ENTER)
    driver.implicitly_wait(10)
    Select(driver.find_element_by_id('view-count1')).select_by_value(value='15')
    time.sleep(1)
    pNum = pageSetting2()
    for j in range(pNum):
        link = driver.find_elements_by_class_name('item_top')
        
        for k in link:
            li = k.find_element_by_tag_name('a').get_attribute('href')
            arr.append(li)
            print(len(arr))
            
        driver.find_element_by_link_text('다음 페이지').send_keys(Keys.ENTER)
        time.sleep(0.5)
    print()
ending(driver)

df = pd.DataFrame(arr)
df.to_csv('link.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067


In [ ]:
#세부정보 링크를 이용한 데이터 크롤링 코드
URL = 'https://www.visitjeju.net/kr/#'
driver = starting(URL) #드라이버 정보 저장하기

df = pd.DataFrame()

for url in link_list:
    driver.get(url)
    name_data = driver.find_element_by_class_name('sub_info_title')
    driver.implicitly_wait(10)
    
    #arr = [] #크롤링 데이터를 저장할 배열
    #info_data = driver.find_elements_by_class_name('real')
    #소개글 크롤링
    #info_arr = ''
    #for i in info_data:
    #    info_arr += i.text.replace('\n','')
    #arr.append(info_arr)

    #안내 크롤링
    i_data = driver.find_elements_by_class_name('add2020_detail_side_info')
    for i in i_data:
        i_arr = i.text.split('\n')
        
    df = df.append(array_return(i_arr, name_data.text), ignore_index=True)
    time.sleep(1)
    
ending(driver)

df.to_csv('link(2).csv', encoding='UTF-8')

In [66]:
#csv 파일 읽어들이는 코드
link = pd.read_csv('link.csv')
link.drop('Unnamed: 0', axis=1, inplace=True)
link_list = link['0'].tolist()

len(link_list)

3482

In [69]:
data = pd.read_excel('./크롤링 데이터/수정된파일.xlsx')
data

,Column1,타입,장소,위치,해시태그,좋아요,찜하기,리뷰,일정 등록,방문했어요,...,유아 서비스,유아 서비스 상세,부대시설 기타,Late체크인여부,공항교통편,장애인전용객실여부,유아 서비스 기타,취급품목,취급품목 기타,택배배송여부
0,0,관광지,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,"['섬속의섬 ', '경관/포토 ', '아이', '맑음']",136,1568,591,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,관광지,성산일출봉(UNESCO 세계자연유산),제주특별자치도 서귀포시 성산읍 일출로 284-12,"['일출 ', '오름 ', '경관/포토', '부모']",213,2713,586,0,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,관광지,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"['숲길 ', '걷기/등산 ', '친구', '커플 ', '흐림 ', '봄']",160,2218,501,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,관광지,카멜리아힐,제주 서귀포시 안덕면 병악로 166,"['경관/포토 ', '커플 ', '아이', '맑음 ', '겨울 ', '우수관광사업체']",127,2295,472,0,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,관광지,협재해수욕장,제주특별자치도 제주시 한림읍 한림로 329-10,"['일몰 ', '해수욕장 ', '액티비티', '아이 ', '맑음 ', '여름']",122,1937,452,0,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477,3477,쇼핑,한림민속오일시장,제주특별자치도 제주시 한림읍 한수풀로4길 10,['전통시장'],4,17,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3478,3478,쇼핑,한마음한라봉,제주특별자치도 제주시 애월읍 애월해안로 32,"['특산품 ', '한라봉']",1,17,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3479,3479,쇼핑,한복담,제주특별자치도 제주시 신대로12길 51,"['커플 ', '친구']",0,4,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3480,3480,쇼핑,함덕오일시장,제주특별자치도 제주시 조천읍 함덕16길 15-13,['전통시장'],0,6,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
URL = 'https://www.visitjeju.net/kr/#'
driver = starting(URL) #드라이버 정보 저장하기
